<a href="https://colab.research.google.com/github/Karlosicolombia/Synthethic-Training-Data-for-Sponswatch-AB/blob/main/Kopia_av_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import PIL
from PIL import Image as img

!pip install pillow
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

directory ='/content/drive/MyDrive/adidas'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab.patches import cv2_imshow

img = cv2.imread(r'/content/drive/MyDrive/Examensarbete/adidaslogo2vit.png',cv2.IMREAD_UNCHANGED)


kernel = np.ones((5, 5), np.uint8)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

Lower_hsv = np.array([51, 120, 0])
Upper_hsv = np.array([255, 255, 255])

# creating the mask by eroding,morphing,
# dilating process
Mask = cv2.inRange(hsv, Lower_hsv, Upper_hsv)
Mask = cv2.erode(Mask, kernel, iterations=1)
Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, kernel)
Mask = cv2.dilate(Mask, kernel, iterations=1)

Mask = cv2.bitwise_not(Mask)
#cv2_imshow(Mask)
#cv2.waitKey(0)

Following are the functions that transform an image with some random augmentations.

In [4]:
def flip(img):
  rand = np.random.uniform(0,1)
  image = img
  if rand > 0.5:
    image = cv2.flip(img, 0)
    return image
  else:
    image = img
    return image

#Mask = augmentationflip(Mask, True)
#cv2_imshow(Mask)




In [5]:
import numpy as np
def rotation(img,rotation):
  rotation = int(np.random.uniform(-rotation,rotation))
  h,w = img.shape[:2]

  matrix = cv2.getRotationMatrix2D((int(w/2),int(h/2)),rotation,0.4 )
  image = cv2.warpAffine(img, matrix, (img.shape[1],img.shape[0]))
  return image

#rotated = rotation(Mask,180)
#cv2_imshow(rotated) 



In [6]:
def randomblur(img):
  Kx = (np.random.randint(0,15))
  Ky = Kx 
  if Kx > 0:
    kernel = (Kx,Ky)
    image = cv2.blur(img,kernel)
    return image
  else: 
    image = img
    return image




In [7]:
def crop(img):
  scaleW = np.random.uniform(0.3,1.5)
  scaleH = np.random.uniform(0.3,1.5)
  
  image = cv2.resize(img, (int(img.shape[1]*scaleW),int(img.shape[0]*scaleH)), interpolation = cv2.INTER_AREA)
  return image







In [8]:
def cutout(img):
  threshold = np.random.uniform(0,1)
  if threshold > 0.1:
    
    endpoint = np.random.uniform(-2,2)

    
    widthS = int(img.shape[1]/2)
    heightS = int(img.shape[0]/2)


    widthE = int(img.shape[1]*endpoint)
    heightE = int(img.shape[0]*endpoint)

    image = cv2.rectangle(img, (widthS, heightS), (widthE, heightE), (0, 0, 0), -1)
    return image
  else:
    return img

In [9]:
import numpy as np
def saturation_image(img):
    version = np.random.randint(0,4)
    saturation = int(np.random.uniform(0,130))
    
    print(version)
    print(saturation)
    version = 1

    if version == 0:
      image = img
      return image
    if version == 1:
      
      return image

    if version == 2:
      image = img[:,saturation,:]
      image = cv2.cvtColor(image,cv2.COLOR_BGR2BGRA)
      return image
    if version == 3:
      image = img[:,:,saturation]
      image = cv2.cvtColor(image,cv2.COLOR_BGR2BGRA)
      return image
    
#img = cv2.imread(r'/content/drive/MyDrive/Examensarbete/adidaslogo2vit.png',cv2.IMREAD_UNCHANGED)
#img = saturation_image(img)   
#cv2_imshow(img)
#cv2.waitKey(0)

The functions below are functions that store files and generates a dataset in a folder.

In [10]:
from PIL import Image as img
import os
import random

def generatecomposite(SizeDataset): 
  
  
  for i in range(SizeDataset):
    os.chdir(r'/content/drive/MyDrive/Examensarbete/train')
    
    with img.open(random.choice(os.listdir(r'/content/drive/MyDrive/Examensarbete/train'))) as image:
      
      image.load()
      
      randomh = int((image.size[1]*0.8) + (image.size[1])*np.random.uniform(-0.8,0))
      randomw = int((image.size[0]*0.5) + (image.size[0]/2)*np.random.uniform(-1,0.1))
      
      
      os.chdir(r'/content/drive/MyDrive/Examensarbete/Dataset')
    with img.open(random.choice(os.listdir(r'/content/drive/MyDrive/Examensarbete/Dataset'))) as overlay:
      overlay.load()
      image.paste(overlay, (randomw,randomh),overlay)
      os.chdir(r'/content/drive/MyDrive/Examensarbete/Synthetic Data')
      image = image.save('synthethic_image_' + str(i)+'.jpeg')
  return



#generatecomposite(20)




In [11]:
import os
def dataset(img,nrsamples):
  path = r'/content/drive/MyDrive/Examensarbete/Dataset'
  name = 'adidas'
  
  for i in range(nrsamples):
    #img = flip(img)
    img = rotation(img,180)
    img = randomblur(img)
    img = crop(img)
    img = cutout(img)
    cv2.imwrite(os.path.join(path , name + str(i) + '.png'), img)
    img = cv2.imread(r'/content/drive/MyDrive/Examensarbete/adidaslogo2vit.png',cv2.IMREAD_UNCHANGED)


img = cv2.imread(r'/content/drive/MyDrive/Examensarbete/adidaslogo2vit.png',cv2.IMREAD_UNCHANGED)
dataset(img,100)


cv2.waitKey(0)


#Lägg till:
#Saturation
#Z
#Noise

-1

Functions that creates a dataset for Deep Image Blending. This model needs three (3) images named "i_target.png","i_source.png" and "i_mask.png". Where target is the background, source is the foreground and mask is the mask of the foreground object. 

In [31]:
from PIL import Image as img
import os
import random


path = r'/content/drive/MyDrive/Examensarbete/' #Location of datasets to choose from.
os.chdir(path) #
def Imageblending(datasetSize):
  for i in range(datasetSize):
    os.chdir(r'/content/drive/MyDrive/Examensarbete/train')
    with img.open(random.choice(os.listdir(r'/content/drive/MyDrive/Examensarbete/train'))) as image:
      image.load()
      os.chdir(r'/content/drive/MyDrive/Examensarbete/Deep Blending Dataset')
      image = image.save(str(i)+'_target.png')
    os.chdir(r'/content/drive/MyDrive/Examensarbete/Dataset')
    with img.open(random.choice(os.listdir(r'/content/drive/MyDrive/Examensarbete/Dataset'))) as image:
      image.load()
      pathtoimage = image.filename 
      os.chdir(r'/content/drive/MyDrive/Examensarbete/Deep Blending Dataset')
      image = image.save(str(i)+'_source.png')
      
      
      image = cv2.imread(r'/content/drive/MyDrive/Examensarbete/Dataset/'+ str(pathtoimage),cv2.IMREAD_UNCHANGED)


      kernel = np.ones((5, 5), np.uint8)
      hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

      Lower_hsv = np.array([51, 120, 0])
      Upper_hsv = np.array([255, 255, 255])

      Mask = cv2.inRange(hsv, Lower_hsv, Upper_hsv)
      Mask = cv2.erode(Mask, kernel, iterations=1)
      Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, kernel)
      Mask = cv2.dilate(Mask, kernel, iterations=1)
      Mask = cv2.bitwise_not(Mask)
      cv2.imwrite(r'/content/drive/MyDrive/Examensarbete/Deep Blending Dataset/'+str(i)+'_mask.png', Mask)
      

Imageblending(2)
